In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
df = pd.read_csv('agentdata/rssi_sl_raw_long.csv')

In [3]:
cols = []
for i in range(64):
    cols.append('Sc' + str(i))

df['Run'] = df.apply(lambda row: int(row.name / 8), axis=1)
df = df[df['Reward'] > 0.0]

obs = []
label = []

for run in df['Run'].unique():
    runData = df[df['Run'] == run]
    myid = runData['Reward'].idxmax()
    myrow = df.loc[myid]
    label.append(myrow['Action'])
    obs.append(myrow[cols].to_numpy())
    #print("max at" + str(myid))
    #print()

In [4]:
df['Dist'].unique()

array([ 0.        ,  0.10045662,  0.20091324,  0.30136986,  0.40182648,
        0.50228311,  0.60273973,  0.70319635,  0.80365297,  0.90410959,
        1.00456621,  1.10502283,  1.20547945,  1.30593607,  1.40639269,
        1.50684932,  1.60730594,  1.70776256,  1.80821918,  1.9086758 ,
        2.00913242,  2.10958904,  2.21004566,  2.31050228,  2.4109589 ,
        2.51141553,  2.61187215,  2.71232877,  2.81278539,  2.91324201,
        3.01369863,  3.11415525,  3.21461187,  3.31506849,  3.41552511,
        3.51598174,  3.61643836,  3.71689498,  3.8173516 ,  3.91780822,
        4.01826484,  4.11872146,  4.21917808,  4.3196347 ,  4.42009132,
        4.52054795,  4.62100457,  4.72146119,  4.82191781,  4.92237443,
        5.02283105,  5.12328767,  5.22374429,  5.32420091,  5.42465753,
        5.52511416,  5.62557078,  5.7260274 ,  5.82648402,  5.92694064,
        6.02739726,  6.12785388,  6.2283105 ,  6.32876712,  6.42922374,
        6.52968037,  6.63013699,  6.73059361,  6.83105023,  6.93

In [5]:
np.array(obs)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
a_size = 8
model = keras.Sequential()
model.add(keras.layers.Dense(64, input_shape=(64,), activation='sigmoid'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(a_size, activation='softmax'))
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
data = np.array(obs)

In [18]:
new_data = data
new_label =[]
new_label.extend(label)

#for i in range(5):
#    for x in data:
        #print(x + np.random.normal(0,0.005,len(x)).reshape(64,))
#        print(len(x))
#        new_data = np.append(new_data, x + np.random.normal(0,0.005,len(x)).reshape(64,1), axis=0)
#        print(new_data.shape)
#    new_label.extend(label)
#new_data = new_data.reshape(int(len(new_data)/64))
new_data.shape

(1099, 64)

In [19]:
len(label)

1099

In [20]:
total_epochs = int(1000)
model.fit(new_data, new_label,
                validation_split=0.2,
                epochs=total_epochs,
                batch_size=32)

Train on 879 samples, validate on 220 samples
Epoch 1/1000
879/879 [==============================] - 0s 339us/sample - loss: 1.9651 - acc: 0.2537 - val_loss: 1.9638 - val_acc: 0.2409
Epoch 2/1000
879/879 [==============================] - 0s 96us/sample - loss: 1.9170 - acc: 0.2685 - val_loss: 1.9246 - val_acc: 0.2409
Epoch 3/1000
879/879 [==============================] - 0s 125us/sample - loss: 1.8848 - acc: 0.2708 - val_loss: 1.9034 - val_acc: 0.2409
Epoch 4/1000
879/879 [==============================] - 0s 112us/sample - loss: 1.8636 - acc: 0.2696 - val_loss: 1.8871 - val_acc: 0.2364
Epoch 5/1000
879/879 [==============================] - 0s 94us/sample - loss: 1.8510 - acc: 0.2753 - val_loss: 1.8677 - val_acc: 0.2500
Epoch 6/1000
879/879 [==============================] - 0s 92us/sample - loss: 1.8343 - acc: 0.2867 - val_loss: 1.8689 - val_acc: 0.2591
Epoch 7/1000
879/879 [==============================] - 0s 78us/sample - loss: 1.8184 - acc: 0.2958 - val_loss: 1.8537 - val_acc:

Epoch 60/1000
879/879 [==============================] - 0s 101us/sample - loss: 1.4378 - acc: 0.3948 - val_loss: 1.7257 - val_acc: 0.3500
Epoch 61/1000
879/879 [==============================] - 0s 96us/sample - loss: 1.4310 - acc: 0.4107 - val_loss: 1.6756 - val_acc: 0.3136
Epoch 62/1000
879/879 [==============================] - 0s 90us/sample - loss: 1.4174 - acc: 0.3948 - val_loss: 1.6859 - val_acc: 0.3182
Epoch 63/1000
879/879 [==============================] - 0s 82us/sample - loss: 1.4277 - acc: 0.4027 - val_loss: 1.7239 - val_acc: 0.3091
Epoch 64/1000
879/879 [==============================] - 0s 77us/sample - loss: 1.4215 - acc: 0.4130 - val_loss: 1.8066 - val_acc: 0.2955
Epoch 65/1000
879/879 [==============================] - 0s 94us/sample - loss: 1.4561 - acc: 0.3982 - val_loss: 1.7616 - val_acc: 0.3227
Epoch 66/1000
879/879 [==============================] - 0s 98us/sample - loss: 1.3859 - acc: 0.4403 - val_loss: 1.7404 - val_acc: 0.3273
Epoch 67/1000
879/879 [==========

879/879 [==============================] - 0s 109us/sample - loss: 1.1072 - acc: 0.5620 - val_loss: 1.8853 - val_acc: 0.3136
Epoch 120/1000
879/879 [==============================] - 0s 108us/sample - loss: 1.1690 - acc: 0.5347 - val_loss: 1.7615 - val_acc: 0.3318
Epoch 121/1000
879/879 [==============================] - 0s 120us/sample - loss: 1.1692 - acc: 0.5108 - val_loss: 1.6680 - val_acc: 0.3818
Epoch 122/1000
879/879 [==============================] - 0s 114us/sample - loss: 1.1204 - acc: 0.5495 - val_loss: 1.6936 - val_acc: 0.3773
Epoch 123/1000
879/879 [==============================] - 0s 93us/sample - loss: 1.1644 - acc: 0.5279 - val_loss: 1.6756 - val_acc: 0.3500
Epoch 124/1000
879/879 [==============================] - 0s 85us/sample - loss: 1.1846 - acc: 0.5199 - val_loss: 2.0178 - val_acc: 0.2455
Epoch 125/1000
879/879 [==============================] - 0s 91us/sample - loss: 1.2603 - acc: 0.4846 - val_loss: 1.7610 - val_acc: 0.3682
Epoch 126/1000
879/879 [==============

Epoch 178/1000
879/879 [==============================] - 0s 105us/sample - loss: 1.0958 - acc: 0.5722 - val_loss: 1.8258 - val_acc: 0.3773
Epoch 179/1000
879/879 [==============================] - 0s 114us/sample - loss: 1.1281 - acc: 0.5438 - val_loss: 1.7348 - val_acc: 0.3909
Epoch 180/1000
879/879 [==============================] - 0s 119us/sample - loss: 1.0424 - acc: 0.5677 - val_loss: 1.6747 - val_acc: 0.3682
Epoch 181/1000
879/879 [==============================] - 0s 108us/sample - loss: 0.9580 - acc: 0.5882 - val_loss: 1.6849 - val_acc: 0.4091
Epoch 182/1000
879/879 [==============================] - 0s 107us/sample - loss: 0.9739 - acc: 0.6121 - val_loss: 1.7308 - val_acc: 0.4500
Epoch 183/1000
879/879 [==============================] - 0s 85us/sample - loss: 1.1139 - acc: 0.5358 - val_loss: 1.9255 - val_acc: 0.3273
Epoch 184/1000
879/879 [==============================] - 0s 89us/sample - loss: 1.0622 - acc: 0.5745 - val_loss: 1.6872 - val_acc: 0.4318
Epoch 185/1000
879/879

Epoch 237/1000
879/879 [==============================] - 0s 94us/sample - loss: 0.7557 - acc: 0.6883 - val_loss: 1.7510 - val_acc: 0.4500
Epoch 238/1000
879/879 [==============================] - 0s 100us/sample - loss: 0.7628 - acc: 0.6780 - val_loss: 1.7459 - val_acc: 0.4591
Epoch 239/1000
879/879 [==============================] - 0s 93us/sample - loss: 0.8192 - acc: 0.6519 - val_loss: 1.9878 - val_acc: 0.3545
Epoch 240/1000
879/879 [==============================] - 0s 79us/sample - loss: 0.9385 - acc: 0.6075 - val_loss: 1.9482 - val_acc: 0.4318
Epoch 241/1000
879/879 [==============================] - 0s 101us/sample - loss: 0.7986 - acc: 0.6712 - val_loss: 1.8306 - val_acc: 0.4000
Epoch 242/1000
879/879 [==============================] - 0s 97us/sample - loss: 0.8043 - acc: 0.6712 - val_loss: 2.0565 - val_acc: 0.4227
Epoch 243/1000
879/879 [==============================] - 0s 95us/sample - loss: 0.8066 - acc: 0.6644 - val_loss: 1.7826 - val_acc: 0.4455
Epoch 244/1000
879/879 [=

...

Epoch 708/1000
879/879 [==============================] - 0s 123us/sample - loss: 0.3817 - acc: 0.8498 - val_loss: 2.3264 - val_acc: 0.5227
Epoch 709/1000
879/879 [==============================] - 0s 115us/sample - loss: 0.3348 - acc: 0.8783 - val_loss: 2.4816 - val_acc: 0.5000
Epoch 710/1000
879/879 [==============================] - 0s 98us/sample - loss: 0.3314 - acc: 0.8760 - val_loss: 2.4716 - val_acc: 0.5045
Epoch 711/1000
879/879 [==============================] - 0s 106us/sample - loss: 0.3238 - acc: 0.8805 - val_loss: 2.3266 - val_acc: 0.5409
Epoch 712/1000
879/879 [==============================] - 0s 97us/sample - loss: 0.3618 - acc: 0.8578 - val_loss: 2.5079 - val_acc: 0.4455
Epoch 713/1000
879/879 [==============================] - 0s 89us/sample - loss: 0.3756 - acc: 0.8578 - val_loss: 2.3366 - val_acc: 0.5364
Epoch 714/1000
879/879 [==============================] - 0s 86us/sample - loss: 0.3682 - acc: 0.8578 - val_loss: 2.5495 - val_acc: 0.4591
Epoch 715/1000
879/879 [

Epoch 767/1000
879/879 [==============================] - 0s 87us/sample - loss: 0.6245 - acc: 0.7304 - val_loss: 2.2868 - val_acc: 0.5273
Epoch 768/1000
879/879 [==============================] - 0s 79us/sample - loss: 0.3844 - acc: 0.8464 - val_loss: 2.1445 - val_acc: 0.5591
Epoch 769/1000
879/879 [==============================] - 0s 83us/sample - loss: 0.3184 - acc: 0.8749 - val_loss: 2.4858 - val_acc: 0.5045
Epoch 770/1000
879/879 [==============================] - 0s 101us/sample - loss: 0.2828 - acc: 0.8965 - val_loss: 2.3683 - val_acc: 0.5545
Epoch 771/1000
879/879 [==============================] - 0s 96us/sample - loss: 0.3253 - acc: 0.8794 - val_loss: 2.3060 - val_acc: 0.5455
Epoch 772/1000
879/879 [==============================] - 0s 99us/sample - loss: 0.4306 - acc: 0.8214 - val_loss: 2.1633 - val_acc: 0.5591
Epoch 773/1000
879/879 [==============================] - 0s 85us/sample - loss: 0.5026 - acc: 0.7918 - val_loss: 2.4419 - val_acc: 0.4955
Epoch 774/1000
879/879 [==

Epoch 826/1000
879/879 [==============================] - 0s 85us/sample - loss: 0.3014 - acc: 0.8771 - val_loss: 2.2558 - val_acc: 0.5636
Epoch 827/1000
879/879 [==============================] - 0s 71us/sample - loss: 0.3694 - acc: 0.8487 - val_loss: 2.5241 - val_acc: 0.5091
Epoch 828/1000
879/879 [==============================] - 0s 82us/sample - loss: 0.2950 - acc: 0.8896 - val_loss: 2.4483 - val_acc: 0.5409
Epoch 829/1000
879/879 [==============================] - 0s 99us/sample - loss: 0.3223 - acc: 0.8680 - val_loss: 2.2836 - val_acc: 0.5955
Epoch 830/1000
879/879 [==============================] - 0s 94us/sample - loss: 0.3770 - acc: 0.8498 - val_loss: 2.4779 - val_acc: 0.5000
Epoch 831/1000
879/879 [==============================] - 0s 88us/sample - loss: 0.3975 - acc: 0.8430 - val_loss: 2.4489 - val_acc: 0.5591
Epoch 832/1000
879/879 [==============================] - 0s 86us/sample - loss: 0.3404 - acc: 0.8623 - val_loss: 2.4404 - val_acc: 0.5182
Epoch 833/1000
879/879 [===

Epoch 885/1000
879/879 [==============================] - 0s 89us/sample - loss: 0.3146 - acc: 0.8851 - val_loss: 2.3864 - val_acc: 0.4636
Epoch 886/1000
879/879 [==============================] - 0s 87us/sample - loss: 0.3965 - acc: 0.8385 - val_loss: 1.9676 - val_acc: 0.5409
Epoch 887/1000
879/879 [==============================] - 0s 85us/sample - loss: 0.3039 - acc: 0.8805 - val_loss: 2.1443 - val_acc: 0.4909
Epoch 888/1000
879/879 [==============================] - 0s 75us/sample - loss: 0.3582 - acc: 0.8441 - val_loss: 2.1894 - val_acc: 0.4864
Epoch 889/1000
879/879 [==============================] - 0s 81us/sample - loss: 0.3365 - acc: 0.8726 - val_loss: 2.7753 - val_acc: 0.4545
Epoch 890/1000
879/879 [==============================] - 0s 86us/sample - loss: 0.4896 - acc: 0.7964 - val_loss: 2.5761 - val_acc: 0.3955
Epoch 891/1000
879/879 [==============================] - 0s 79us/sample - loss: 0.4419 - acc: 0.8066 - val_loss: 2.1856 - val_acc: 0.4727
Epoch 892/1000
879/879 [===

Epoch 944/1000
879/879 [==============================] - 0s 89us/sample - loss: 0.2794 - acc: 0.8976 - val_loss: 1.9388 - val_acc: 0.5727
Epoch 945/1000
879/879 [==============================] - 0s 92us/sample - loss: 0.3308 - acc: 0.8453 - val_loss: 2.0353 - val_acc: 0.5955
Epoch 946/1000
879/879 [==============================] - 0s 79us/sample - loss: 0.3318 - acc: 0.8749 - val_loss: 2.5340 - val_acc: 0.5227
Epoch 947/1000
879/879 [==============================] - 0s 81us/sample - loss: 0.3540 - acc: 0.8601 - val_loss: 1.9519 - val_acc: 0.5409
Epoch 948/1000
879/879 [==============================] - 0s 73us/sample - loss: 0.3247 - acc: 0.8646 - val_loss: 2.5054 - val_acc: 0.5000
Epoch 949/1000
879/879 [==============================] - 0s 93us/sample - loss: 0.5482 - acc: 0.8032 - val_loss: 1.9494 - val_acc: 0.5227
Epoch 950/1000
879/879 [==============================] - 0s 82us/sample - loss: 0.5489 - acc: 0.7770 - val_loss: 2.5185 - val_acc: 0.4818
Epoch 951/1000
879/879 [===

[ 0.          0.          0.          0.          0.          0.
 39.70596313 44.78884888 35.22101593 38.83454895 39.37739182 41.10073471
 35.25659943 42.16393661 41.81136322 40.18881989 41.75072098 42.05091476
 41.734375   33.74575806 39.66823959 34.89291763 41.79295731 41.55587006
 35.26836395 41.10770798 39.85906219 38.62493896 38.89492798 40.29067993
 42.55525589 42.04333878  0.         28.88636971 41.78952789 40.03998184
 41.5358696  35.46432114 27.45736313 41.25907898 28.86631966 39.6586113
 41.75383759 41.13529205 38.73374557 39.86402512 38.89834213 39.45638657
 34.66478729 38.91122818 39.4909935  33.53464508 42.13908005 40.41591644
 43.51588058 41.89361954 41.58200455 41.37800217 40.10470581  0.
  0.          0.          0.          0.        ]
[ 0.          0.          0.          0.          0.          0.
 37.77603149 41.19931412 42.61149597 41.21717834 38.65360641 39.52934265
 39.55318069 34.95041656 42.50469208 33.66572571 41.33837891 23.80127335
 34.1560936  41.67356491 2